In [1]:
#Importar librerías
from vivienda_cdmx.vivicdmx import lamudi,easybroker
import pandas as pd
import time
import random
from matplotlib import pyplot as plt

#### Scrapeo de datos de los portales inmobiliarios

In [2]:
# Listado
portales = [ easybroker, lamudi]
# df vacía
df = pd.DataFrame()

# proceso de scrapeo
for portal in portales:
    try:
        # Tiempo de inicio
        print("Proceso iniciado a las ", time.strftime("%H:%M:%S"),"del día", time.strftime("%d/%m/%y"))
        
        # Ejecuta la función de scraping
        df_portal = portal()  
        df = pd.concat([df, df_portal])
        
        # Records
        print("Número de registros: ", len(df))
        
    except Exception as e:
        # Si ocurre un error, lo imprime
        print(f"Ocurrió un error: {e}")
    
    finally:
        
        print("Proceso terminado a las ", time.strftime("%H:%M:%S"),"del día", time.strftime("%d/%m/%y"))
        print("--------------------------------------------------")
        
        # Sleep for a random time before the next iteration
        time.sleep(random.randint(1, 3))

    
df

Proceso iniciado a las  11:35:28 del día 07/04/24


Scrapeando Easybroker: 100%|██████████| 100/100 [07:40<00:00,  4.60s/it]


Número de registros:  2375
Proceso terminado a las  11:43:09 del día 07/04/24
--------------------------------------------------
Proceso iniciado a las  11:43:11 del día 07/04/24


Scrapeando Lamudi: 100%|██████████| 100/100 [05:51<00:00,  3.51s/it]


Ocurrió un error: expected string or bytes-like object
Proceso terminado a las  11:49:02 del día 07/04/24
--------------------------------------------------


,oferta,precio,mts_const,direccion,bathrooms,recamaras,lat,lon,fuente,fecha_consulta
0,jardines del pedregal residencia de lujo con e...,44500000.00,988.0,"Casa en Jardines del Pedregal, Álvaro Obregón",5.0,5.0,19.3173164,-99.2032124,easybroker,2024-04-07 11:43:09.083540
1,casa en venta en jardines del pedregal,25000000.00,640.0,"Casa en Jardines del Pedregal, Álvaro Obregón",4.0,5.0,19.3285645,-99.2001782,easybroker,2024-04-07 11:43:09.083540
2,casa ubicada en el triangulo dorado 4 rec,39541920.00,630.0,"Casa en Lomas de Chapultepec, Miguel Hidalgo",4.0,4.0,19.4249374,-99.2134442,easybroker,2024-04-07 11:43:09.083540
3,venta casa en panamericana gustavo a. madero ...,4200000.00,280.0,"Casa en Ampliación Panamericana, Gustavo A. Ma...",2.0,5.0,19.520674,-99.1493664,easybroker,2024-04-07 11:43:09.083540
4,como terreno 467 m2,60000000.00,435.0,"Casa en Polanco, Miguel Hidalgo",4.0,4.0,19.4356762643,-99.1987753003,easybroker,2024-04-07 11:43:09.083540
...,...,...,...,...,...,...,...,...,...,...
2370,casa en condominio y calle cerrada en venta,55000000.00,950.0,"Casa en Jardines del Pedregal, Álvaro Obregón",3.0,3.0,19.3309303,-99.2052473,easybroker,2024-04-07 11:43:09.083540
2371,casa sola en venta en el pedregal,27500000.00,746.0,"Casa en Jardines del Pedregal, Álvaro Obregón",5.0,5.0,19.3177586,-99.1992804,easybroker,2024-04-07 11:43:09.083540
2372,casa en venta 4 recamaras en calle cerrada arq...,32500000.00,656.0,"Casa en Jardines del Pedregal, Álvaro Obregón",4.0,4.0,19.3283277,-99.2025838,easybroker,2024-04-07 11:43:09.083540
2373,super oportunidad baja de precio casa en lomas...,9480000.00,361.0,"Casa en San Jerónimo Lídice, La Magdalena Cont...",3.0,3.0,19.3209194,-99.2364104,easybroker,2024-04-07 11:43:09.083540


In [3]:
df["fuente"].unique()

array(['easybroker'], dtype=object)

In [5]:
#Salvar datos sin limpiar
df.to_csv("vivienda_yucatan_sucio.csv",index=False,encoding="utf-8-sig")
push_request(df, "viviyuca_sucia.csv", "Se envía scrapeo sucio", "data")

Archivo actualizado con éxito


#### Limpieza de datos

In [6]:
def limpia_datos(df):
    df = df.reset_index(drop=True)
   
    #Eliminar registros con precio 0 o nan
    df=df[df['precio']>0]
    df=df[df['precio'].notna()]
    #Eliminar registros que en oferta contengan "terreno"
    df=df[~df['oferta'].str.contains('terreno')]
    df=df[~df['oferta'].str.contains('renta')]
    df=df[~df['oferta'].str.contains('remodelar')]
    df=df[~df['oferta'].str.contains('hectareas')]
    #Si la fuente es goodlers, sacar el promedio de precio_min y precio_max y ponerlo en precio
    #Eliminar registros con misma oferta y mismo precio
    df=df.drop_duplicates(subset=['oferta','precio'],keep='first')

    #Calcular precio por metro cuadrado
    df['precio_m2'] = df['precio'] / df['mts_const']

    return df

In [7]:
df_limpia=limpia_datos(df)
df_limpia

,desarrollador,oferta,fuente,localidad,precio_min,precio_max,fecha_consulta,lat,lon,precio,...,cochera,alberca,vigilancia,paddle,casa_club,privada,size,superficie,colloc,precio_m2
0,Pedregales de Misné balám,casas en misne balam,goodlers,None,1607000.0,3311170.0,2024-03-17 15:48:45.298945,21.171149,-89.606722,2459085.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,KOBÁ Townhouses,townhouses y casas en temozon norte,goodlers,temozon norte,3600000.0,3600000.0,2024-03-17 15:48:45.298945,21.066644,-89.607698,3600000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temozon norte,NaN
2,Zensia Residencial,casas en conkal,goodlers,None,2360000.0,3040000.0,2024-03-17 15:48:45.298945,21.064648,-89.527726,2700000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Kenatta Luxury Villas,casas y townhouses en temozon norte,goodlers,temozon norte,4500000.0,4500000.0,2024-03-17 15:48:45.298945,21.054118,-89.609704,4500000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temozon norte,NaN
4,Enuma Urban Homes,casas en temozon norte,goodlers,temozon norte,4329593.0,4927550.0,2024-03-17 15:48:45.298945,21.059675,-89.613845,4628571.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,temozon norte,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8354,NaN,pensiones vi etapa,inmuebles24,pensiones,NaN,NaN,2024-03-17 16:36:18.423800,NaN,NaN,2000000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,160,pensiones,NaN
8356,NaN,venta de residencia de lujo killil en yucatan ...,inmuebles24,yucatan country club,NaN,NaN,2024-03-17 16:36:18.423800,NaN,NaN,33150000.0,...,1.0,1.0,0.0,0.0,0.0,0.0,NaN,947,yucatan country club,NaN
8358,NaN,carrt. dzibilchaltun,inmuebles24,dzibilchaltun,NaN,NaN,2024-03-17 16:36:18.423800,NaN,NaN,4187000.0,...,1.0,1.0,1.0,0.0,1.0,1.0,NaN,136,dzibilchaltun,NaN
8361,NaN,calle 45 numero 544 la ciudadela ciudad caucel,inmuebles24,ciudad caucel,NaN,NaN,2024-03-17 16:36:18.423800,NaN,NaN,1400000.0,...,0.0,0.0,0.0,0.0,0.0,1.0,NaN,163,ciudad caucel,NaN


In [8]:
df_limpia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4399 entries, 0 to 8362
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   desarrollador   57 non-null     object        
 1   oferta          4399 non-null   object        
 2   fuente          4399 non-null   object        
 3   localidad       2463 non-null   object        
 4   precio_min      57 non-null     float64       
 5   precio_max      57 non-null     float64       
 6   fecha_consulta  4399 non-null   datetime64[ns]
 7   lat             1620 non-null   object        
 8   lon             1620 non-null   object        
 9   precio          4399 non-null   float64       
 10  mts_const       1487 non-null   float64       
 11  direccion       23 non-null     object        
 12  bathrooms       2488 non-null   object        
 13  recamaras       2770 non-null   object        
 14  colonia         1254 non-null   object        
 15  bedrooms 